In [ ]:
%gui qt
%matplotlib qt

import os
from pprint import pprint
from importlib import reload

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from pyqtgraph.Qt import QtGui, QtCore

In [ ]:
import qcodes as qc
from qcodes import ParamSpec, new_data_set
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.database import initialise_database

In [ ]:
from plottr.apps import autoplot

from plottr.data import qcodes_dataset; reload(qcodes_dataset)
from plottr.data.qcodes_dataset import (get_runs_from_db_as_dataframe,
                                        get_ds_info_from_path,
                                        datadict_from_path_and_run_id,
                                        QCodesDBInspector)

In [ ]:
# DB_PATH = "./test_10k_sets.db"
DB_PATH = "../testdata/experiments_BF2.db"

# Creating qcodes test data

## make large number of datasets

In [ ]:
def test_data(nx, ny):
    x = np.arange(nx, dtype=np.float64)
    y = np.arange(ny, dtype=np.float64)
    xx, yy = np.meshgrid(x, y)
    zz = xx * yy
    
    return xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)

def insert_data(name='dummy_data', nx=10, ny=10):
    ds = new_data_set('dummy_data', 
                      specs=[ParamSpec('x', 'array', unit='V'),
                             ParamSpec('y', 'array', unit='A'),
                             ParamSpec('z1', 'array', unit='C', depends_on=['x', 'y']),
                             ParamSpec('z2', 'array', unit='C', depends_on=['x', 'y'])])
    x, y, z = test_data(nx, ny)
    ds.add_result(dict(x=x, y=y, z1=z, z2=z))
    ds.mark_complete()
    
    return ds.run_id

def make_test_db(nruns=100, nx=100, ny=100):
    qc.config.core.db_location = DB_PATH
    initialise_database()
    
    for i in range(nruns):
        print(f'Inserting run {i+1}/{nruns}' + 10*' ', end='\r')
        if not i%25:
            new_experiment(name=f'test_exp_{i//25}', sample_name="no sample")
    
        insert_data(nx, ny)

In [ ]:
# make_test_db(nruns=10000)

## Make and retrieve data

### Trivial case. softsweep with very simple shape.

In [ ]:
def make_numeric_test_ds():
    ds = new_data_set(
        'dummy_data', 
        specs=[ParamSpec('x', 'numeric', unit='V'),
               ParamSpec('y', 'numeric', unit='A'),
               ParamSpec('z1', 'numeric', unit='C', depends_on=['x', 'y']),
               ParamSpec('z2', 'numeric', unit='C', depends_on=['x', 'y'])])
    
    for x in range(5):
        for y in range(5):
            ds.add_result(dict(x=x, y=y, z1=np.random.rand(), z2=np.random.rand()))
    ds.mark_complete()
    return ds.run_id

DB_PATH = "./testing.db"
qc.config.core.db_location = DB_PATH
initialise_database()
new_experiment(name='test_trivial_insertion', sample_name='no sample')
run_id = make_numeric_test_ds()

In [ ]:
ds = DataSet(path_to_db=DB_PATH, run_id=run_id)
pprint(ds.get_data('x', 'y', 'z1', 'z2'))

### Two dependents, but one only dependent on the outer axis.

In [ ]:
def make_numeric_test_ds():
    ds = new_data_set(
        'dummy_data', 
        specs=[ParamSpec('x', 'numeric', unit='V'),
               ParamSpec('y', 'numeric', unit='A'),
               ParamSpec('z1', 'numeric', unit='C', depends_on=['x', 'y']),
               ParamSpec('z2', 'numeric', unit='C', depends_on=['x'])])
    
    for x in range(5):
        for y in range(5):
            results = dict(x=x, y=y, z1=np.random.rand())
            if y%5 == 0:
                results['z2'] = np.random.rand()
            ds.add_result(results)
    
    ds.mark_complete()
    return ds.run_id

DB_PATH = "./testing.db"
qc.config.core.db_location = DB_PATH
initialise_database()
new_experiment(name='test_insertion_with_different_frequency', sample_name='no sample')
run_id = make_numeric_test_ds()

In [ ]:
ds = DataSet(path_to_db=DB_PATH, run_id=run_id)
pprint(ds.get_data('x', 'y', 'z1', 'z2'))

### Array data for z1 and the inner sweep param

In [ ]:
def make_numeric_test_ds():
    ds = new_data_set(
        'dummy_data', 
        specs=[ParamSpec('x', 'numeric', unit='V'),
               ParamSpec('y', 'array', unit='A'),
               ParamSpec('z1', 'array', unit='C', depends_on=['x', 'y']),
               ParamSpec('z2', 'numeric', unit='C', depends_on=['x'])])
    
    for x in range(5):
        y = np.arange(5.0)
        results = dict(x=x, y=y, z1=np.random.rand(5), z2=np.random.rand())
        ds.add_result(results)
    
    ds.mark_complete()
    return ds.run_id

DB_PATH = "./testing.db"
qc.config.core.db_location = DB_PATH
initialise_database()
new_experiment(name='test_insertion_with_both_array_and_numeric', sample_name='no sample')
run_id = make_numeric_test_ds()

In [ ]:
ds = DataSet(path_to_db=DB_PATH, run_id=run_id)
data = qcodes_dataset.ds_to_datadict(ds)

In [ ]:
pprint(data)

# DB Inspector

In [ ]:
win = QCodesDBInspector(dbPath=DB_PATH)
win.show()

# Autoplotting a qcodes dataset

In [ ]:
data = datadict_from_path_and_run_id(path=DB_PATH, run_id=90)
fc, plotwin = autoplot.autoplot(inputData=data, log=True)

# Autoplotting with qcodes loader node

In [ ]:
fc, plotwin = autoplot.autoplotQcodesDataset(log=True)
dsloader = fc.nodes()['QCodesDSLoader.0']
dsloader.pathAndId = DB_PATH, 90

In [ ]:
fc2, plotwin2 = autoplot.autoplotQcodesDataset(log=True)
dsloader = fc2.nodes()['QCodesDSLoader.0']
dsloader.pathAndId = DB_PATH, 115

# Scratch

In [ ]:
np.product((1,2,3))

In [ ]:
np.arange(27).reshape(3,3,3).reshape(-1)